In [ ]:
import json
import numpy as np

def stable_softmax(x):
    z = x - max(x)
    numerator = np.exp(z)
    denominator = np.sum(numerator)
    softmax = numerator/denominator

    return softmax

In [ ]:
with open('fa_out.json','r',encoding='utf-8-sig') as fp:
    txt = fp.read()

In [ ]:
data = json.loads(txt)

In [ ]:
data.keys()

In [ ]:
for p in data['players']:
    p['freeAgentMood']= 30*[0]
for t in data['gameAttributes']:
    if t['key'] == 'salaryCap':
        cap = t['value']
    if t['key'] == 'minContract':
        minS = t['value']
    if t['key'] == 'maxContract':
        maxS = t['value']

In [ ]:
RAND_NUM = 256 # power of 2 plz
RN = 256 - 1 # mod 2 into hash
rvec = np.random.randn(256)*2

TEMP = 1.0
SCALE_UP = 1.05
SCALE_DOWN = 0.95
ROUNDS = 25

In [ ]:
players = {}
for p in data['players']:
    if p['tid'] != -1:
        continue
        
    # i dunno, good enough?
    val = p['ratings'][-1]['ovr']*0.5 + p['ratings'][-1]['pot']*0.5
    
    # basically ws/48 prediction from OVR
    val_or = (1/209.33) * (max(val,29.14) - 29.14) ** 2
    
    apy = p['contract']['amount']
    players[p['pid']] = (val_or,apy)


In [ ]:
from collections import defaultdict
teams = defaultdict(list)
for p in data['players']:
    if p['tid'] >= 0:
        teams[p['tid']].append(p['contract']['amount'])


In [ ]:
team_totals = {k:sum(v) for k,v in teams.items()}

In [ ]:
for i in range(ROUNDS):
    # teams bid on players

    bids = defaultdict(int)
    for t,c in team_totals.items():
        budget = cap-c
        selected = set()
        while budget > 0:
            # get valid players
            valid_p = [(p[0]+rvec[hash((pid,t))&RN],pid) for pid,p in players.items() if p[1]<=budget and pid not in selected]

            if len(valid_p) > 0:
                # sample one
                vpa = np.array(valid_p)
                probs = stable_softmax(vpa[:,0]*TEMP)
                pid = int(np.random.choice(vpa[:,1],1,p=probs))

                # act accordingly
                selected.add(pid)
                bids[pid] = 1 + bids[pid]
                budget -= players[pid][1]
            else:
                break
                
    # players adjust expectations
    for pid,v in players.items():
        if pid in bids and bids[pid] > 1:
            players[pid]  = (v[0],min(maxS,v[1]*SCALE_UP))
        elif bids[pid] == 0:
            players[pid] = (v[0],max(minS,v[1]*SCALE_DOWN))
            #print(players[pid],pid)

In [ ]:
res = []
for p in data['players']:
    if p['tid'] != -1:
        continue
    res.append((p['contract']['amount'],players[p['pid']][1]))

In [ ]:
import matplotlib.pyplot as plt
plt.subplot(1,2,1)
plt.hist(vpa[:,0])
plt.subplot(1,2,2)
plt.hist(probs)

In [ ]:
plt.scatter(np.array(res)[:,0],np.array(res)[:,1])

In [ ]:
for i,e in enumerate(data['players']):
    if e['pid'] in players:
        data['players'][i]['contract']['amount'] = players[e['pid']][1]

In [ ]:
with open('fa_fixed.json','w',encoding='utf-8-sig') as fp:
    json.dump(data,fp)